In [171]:
import numpy as np
import random
import scipy.sparse
import sys
import time

from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession
from pyspark.mllib.clustering import LDA, LDAModel
from pyspark.mllib.linalg import Vectors

In [172]:
conf = SparkConf().setMaster("local[*]").setAll([
                                   ('spark.executor.memory', '16g'),  # find
                                   ('spark.driver.memory','8g'), # your
                                   ('spark.driver.maxResultSize', '4G') # setup
                                  ])
# create the session
spark = SparkSession.builder.config(conf=conf).getOrCreate()

# create the context
sc = spark.sparkContext

In [173]:
S = scipy.sparse.load_npz('/dlabdata1/youtube_large/olam/matrices/S_final.npz')

In [202]:
data = []

In [203]:
def get_dict_for_row(row):
    ''''''
    tmp_dict = {}
    for key, value in row:
        tmp_dict[key[1]] = value
        
    return Vectors.sparse(len(tmp_dict), tmp_dict)

In [204]:
for i in range(S.shape[0]):
    
    # Data is a list of list of the following elems : index of doc and a bag-of-word sparse Vector
    data.append([i, get_dict_for_row(S.getrow(i).todok().items())])
    

Duration for transforming to rdd : 0.044542789459228516


In [205]:
corpus = sc.parallelize(data)

In [206]:
model = LDA.train(corpus, k=50, seed=1)

In [213]:
model.save(sc, '/dlabdata1/youtube_large/olam/lda_sparkModel')

In [222]:
same_model = LDAModel.load(sc, '/dlabdata1/youtube_large/olam/lda_sparkModel')

In [226]:
model.describeTopics()[0]

([214313, 389685, 702165, 671629, 420134, 544917],
 [0.266132635069856,
  0.04908022772804397,
  0.04380826578831293,
  0.04196875751770345,
  0.031857012470995164,
  0.015664738377080007])

In [227]:
same_model.describeTopics()[0]

([214313, 389685, 702165, 671629, 420134, 544917],
 [0.266132635069856,
  0.04908022772804397,
  0.04380826578831293,
  0.04196875751770345,
  0.031857012470995164,
  0.015664738377080007])